Get From https://github.com/BlaBlaBlazzz/Fake_News_classification/blob/main/Fake_news_identification.ipynb

Because he said there is a performance issue, so I copy it and test.

In [1]:
import os
from tqdm import tqdm
import glob
import pandas as pd
import numpy as np
glob.glob("*.csv.zip")

['sample_submission.csv.zip', 'test.csv.zip', 'train.csv.zip']

In [2]:
#os.system("unzip train.csv.zip")
#os.system("unzip test.csv.zip")

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv("sample_submission.csv")

train = train.reset_index()
cols = ['title1_zh', 'title2_zh', 'label']
train = train.loc[:,cols]
train.columns = ['text_a', 'text_b', 'label']

train.to_csv("train.tsv", sep="\t", index=False)
train.head()

,text_a,text_b,label
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,unrelated
4,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,agreed


In [3]:
train.label.value_counts() / len(train)

unrelated    0.684173
agreed       0.290040
disagreed    0.025787
Name: label, dtype: float64

In [4]:
pip install transformers tqdm boto3 requests regex -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
from torch.utils.data import Dataset
import tensorflow
import torch
torch.cuda.is_available()
import transformers
from transformers import BertTokenizer
#from pytorch_transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

C:\Users\BlaBlaBla\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\BlaBlaBla\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\BlaBlaBla\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\BlaBlaBla\AppData\Roaming\Pytho

In [6]:
from torch.utils.data import Dataset

class FakeNewsIdensification(Dataset):
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]
        self.mode = mode
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {"agreed":0, "disagreed":1, "unrelated":2}
        self.tokenizer = tokenizer
  

    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, text_b = self.df.iloc[idx,:2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx,:].values
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)

    
        word_pieces = ["[CLS]"]
        token_a = self.tokenizer.tokenize(text_a)
        word_pieces += token_a
        word_pieces += ["[SEP]"]
        len_a = len(word_pieces)

        token_b = self.tokenizer.tokenize(text_b)
        word_pieces += token_b
        len_b = len(word_pieces) - len_a

        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        segments_tensor = torch.tensor([0]*len_a + [1]*len_b, dtype=torch.long)

        return (tokens_tensor, segments_tensor, label_tensor)

    def __len__(self):
        return self.len

trainset = FakeNewsIdensification("train", tokenizer=tokenizer)

In [7]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def mini_batch(samples):
    token_tensors = [s[0] for s in samples]
    segment_tensors = [s[1] for s in samples]

    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = none

    token_tensors = pad_sequence(token_tensors, batch_first=True)
    segment_tensors = pad_sequence(segment_tensors, batch_first=True)

    #Mask_tensor
    mask_tensors = torch.zeros(token_tensors.shape, dtype=torch.long)
    mask_tensors = mask_tensors.masked_fill(token_tensors != 0, 1)

    return token_tensors, segment_tensors, mask_tensors, label_ids

BATCH_SIZE = 64
if __name__ == '__main__':
    trainloader = DataLoader(
        trainset, 
        batch_size = BATCH_SIZE,
        collate_fn = mini_batch,
        num_workers = 0,
        pin_memory = True
    )

In [8]:
from transformers import BertForSequenceClassification, BertPreTrainedModel
from IPython.display import display, clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

clear_output()

model.config

BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

In [9]:
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels=3):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)  #config -> bert的預訓練參數
    
        #hidden_dropout_prob = 0.1
        self.dropout(config.hidden_dropout_prob)
    
        #hidden_size=768
        self.linear(config.hidden_size, num_labels)

    
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, label=None):
        output = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)  #pooled_output -> 訓練bert輸出
    
        logits = self.classifier(pooled_output)

        if labels is not None:
            self.loss = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
    
        elif self.output_attentions:
            return all_attentions, logits
    
        return logits
    
#if __name__ == '__main__':
#    dataloader = DataLoader(trainset, num_workers=0)
#    for _ in tqdm(dataloader):
#        pass

In [10]:
def get_prediction(model, dataloader, compute_acc=None):
    predictions = None
    correct = 0
    total = 0
    
    
    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
                
    token_tensors , segment_tensors, mask_tensors = data[:3]
    
    outputs = model(input_ids = token_tensors, token_type_ids = segment_tensors,
                  attention_mask = mask_tensors)
    
    logits = outputs[0]
    _, prediction = torch.max(logits.data, 1)
    
    # compute_acc == True
    if compute_acc:
        labels = data[3]
        total += labels.size(0)
        correct = (prediction==labels).sum().item()
    
    if predictions is None:
        predictions = prediction
    else:
        predictions = torch.cat(predictions, prediction)
        
        
    if compute_acc:
        accuracy = correct/total
        return prediction, accuracy
    else:
        return prediction
    
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

model = model.to(device)
_, acc = get_prediction(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cuda:0
classification acc: 0.175


In [ ]:
model.eval()
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

epochs = 5
for epoch in range(epochs):
    
    epoch_loss = 0
    if __name__ == '__main__':
        for data in trainloader:
            token_tensors, segment_tensors, mask_tensors, labels = [t.to(device) for t in data]

            optimizer.zero_grad()

            
            output = model(input_ids = token_tensors, token_type_ids = segment_tensors,
                               attention_mask = mask_tensors, labels = labels)

            loss = output[0]

            loss = loss.requires_grad_()

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            _, acc = get_prediction(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %(epoch + 1, epoch_loss, acc))